In [1]:
pip install xlwings pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import xlwings as xw
import pandas as pd

# 현재 열려있는 엑셀 파일 열기
wb = xw.apps.active.books.active

# 병합 결과 저장할 리스트
merged_data = []
row_offset = 0  # 출력될 행의 위치를 관리

# 새 시트 생성 (기존에 있으면 삭제 후 새로 생성)
if '병합시트' in [sheet.name for sheet in wb.sheets]:
    wb.sheets['병합시트'].delete()
merge_sheet = wb.sheets.add('병합시트')

for sheet in wb.sheets:
    if sheet.name == '병합시트':
        continue

    # 데이터 읽기
    df = sheet.used_range.options(pd.DataFrame, header=0, index=False).value

    if df is None or df.empty:
        continue

    # 병합 대상 시트 이름 표시
    merge_sheet.range((row_offset + 1, 1)).value = f'[{sheet.name}] 시트 내용'
    row_offset += 1

    # 데이터 기록
    merge_sheet.range((row_offset + 1, 1)).value = df.values.tolist()
    row_offset += len(df.index) + 3  # 데이터 다음에 3줄 간격 추가

print("시트 병합이 완료되었습니다.")

시트 병합이 완료되었습니다.


In [3]:
import xlwings as xw
import pandas as pd

# 현재 열려있는 엑셀 파일 열기
wb = xw.apps.active.books.active

# 병합 결과 저장할 리스트
merged_data = []
row_offset = 0  # 출력될 행의 위치를 관리

# 새 시트 생성 (기존에 있으면 삭제 후 새로 생성)
if '병합시트' in [sheet.name for sheet in wb.sheets]:
    wb.sheets['병합시트'].delete()
merge_sheet = wb.sheets.add('병합시트')

# 전체 컬럼 집합 만들기
all_columns = set()
sheet_data = {}

for sheet in wb.sheets:
    if sheet.name == '병합시트':
        continue

    df = sheet.used_range.options(pd.DataFrame, header=0, index=False).value
    if df is None or df.empty:
        continue

    df.columns = df.columns.astype(str)  # 컬럼명을 문자열로
    all_columns.update(df.columns)  # 전체 컬럼 목록에 추가
    sheet_data[sheet.name] = df

# 전체 컬럼 정렬
all_columns = sorted(all_columns)

# 시트별로 병합
for sheet_name, df in sheet_data.items():
    df = df.copy()

    # 누락된 컬럼은 빈 값으로 추가
    for col in all_columns:
        if col not in df.columns:
            df[col] = ""

    # 전체 컬럼 순서로 정렬
    df = df[all_columns]

    # 병합 대상 시트 이름 표시
    merge_sheet.range((row_offset + 1, 1)).value = f'[{sheet_name}] 시트 내용'
    row_offset += 1

    # 헤더 작성
    merge_sheet.range((row_offset + 1, 1)).value = [all_columns]
    row_offset += 1

    # 값 작성 (to_list() 안전처리 포함)
    data_rows = df.values.tolist()
    if data_rows:
        merge_sheet.range((row_offset + 1, 1)).value = data_rows
        row_offset += len(data_rows)

    # 시트 간격
    row_offset += 3

print("✅ 시트 병합이 완료되었습니다.")

com_error: (-2147352567, '예외가 발생했습니다.', (0, None, None, None, 0, -2146827284), None)

In [1]:
import xlwings as xw
import pandas as pd
from pathlib import Path
from datetime import datetime

# 저장 파일 경로 설정
today_str = datetime.today().strftime('%Y%m%d')
save_path = Path.cwd() / f'병합완료_{today_str}.xlsx'

# 새로운 엑셀 파일 생성
app = xw.App(visible=False)
new_wb = app.books.add()
merge_sheet = new_wb.sheets[0]
merge_sheet.name = "병합시트"

# 원본 엑셀 파일 접근
source_wb = xw.apps.active.books.active

row_offset = 0  # 병합 시트의 현재 행 위치

# 각 시트의 데이터를 읽어와 병합
for sheet in source_wb.sheets:
    if sheet.name.lower() == '병합시트':
        continue  # 병합 시트는 제외

    try:
        # 시트의 데이터를 DataFrame으로 읽어오기
        df = sheet.range('A1').expand('table').options(pd.DataFrame, header=0, index=False).value

        if df is None or df.empty:
            continue  # 데이터가 없는 시트는 건너뜀

        # 시트 이름을 병합 시트에 표시
        merge_sheet.range((row_offset + 1, 1)).value = f"[{sheet.name}] 시트 내용"
        row_offset += 1

        # 데이터의 헤더와 내용을 병합 시트에 작성
        merge_sheet.range((row_offset + 1, 1)).value = [df.columns.tolist()]
        merge_sheet.range((row_offset + 2, 1)).value = df.values.tolist()

        # 줄 바꿈 적용
        total_rows = len(df)
        total_cols = len(df.columns)
        merge_sheet.range((row_offset + 1, 1), (row_offset + total_rows + 1, total_cols)).api.WrapText = True

        row_offset += total_rows + 4  # 다음 시트의 데이터를 위해 행 위치 조정

    except Exception as e:
        print(f"❌ {sheet.name} 시트에서 오류 발생: {e}")

# 병합된 데이터를 새로운 파일에 저장
new_wb.save(str(save_path))
new_wb.close()
app.quit()

print(f"✅ 병합된 새 파일 저장 완료: {save_path}")

✅ 병합된 새 파일 저장 완료: C:\Users\User\Desktop\jupytertest\병합완료_20250422.xlsx


In [3]:
import xlwings as xw
from pathlib import Path
from datetime import datetime

# 저장 파일 경로 설정
today_str = datetime.today().strftime('%Y%m%d')
save_path = Path.cwd() / f'병합완료_{today_str}.xlsx'

# 새로운 엑셀 파일 생성
app = xw.App(visible=False)
new_wb = app.books.add()
merge_sheet = new_wb.sheets[0]
merge_sheet.name = "병합시트"

# 원본 엑셀 파일 접근
source_wb = xw.apps.active.books.active

row_offset = 0  # 병합 시트의 현재 행 위치

# 각 시트의 데이터를 읽어와 병합
for sheet in source_wb.sheets:
    if sheet.name.lower() == '병합시트':
        continue  # 병합 시트는 제외

    try:
        # 시트의 데이터를 리스트로 읽어오기
        data = sheet.range('A1').expand('table').value

        if not data:
            continue  # 데이터가 없는 시트는 건너뜀

        # 시트 이름을 병합 시트에 표시
        merge_sheet.range((row_offset + 1, 1)).value = f"[{sheet.name}] 시트 내용"
        row_offset += 1

        # 데이터의 헤더와 내용을 병합 시트에 작성
        merge_sheet.range((row_offset + 1, 1)).value = data

        # 줄 바꿈 적용
        total_rows = len(data)
        total_cols = len(data[0]) if data else 0
        merge_sheet.range((row_offset + 1, 1), (row_offset + total_rows, total_cols)).api.WrapText = True

        row_offset += total_rows + 2  # 다음 시트의 데이터를 위해 행 위치 조정

    except Exception as e:
        print(f"❌ {sheet.name} 시트에서 오류 발생: {e}")

# 병합된 데이터를 새로운 파일에 저장
new_wb.save(str(save_path))
new_wb.close()
app.quit()

print(f"✅ 병합된 새 파일 저장 완료: {save_path}")

✅ 병합된 새 파일 저장 완료: C:\Users\User\Desktop\jupytertest\병합완료_20250422.xlsx
